# ID2214/FID3214 Assignment 3 Group no. ID2214 – 5
### Project members:
Lukas Olenborg,

Charlotte Jacquet,

Isabella Rositi

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Filepath to shared drive (you have to add the 'Programming for Data Science' folder to your drive)
#filepath = '/content/drive/MyDrive/Programming for Data Science/Assignment 3/'

# Filepath to your own drive folder and files if above doesnt work
#filepath = '/content/drive/MyDrive/Colab Notebooks/ID2214/'

In [ ]:
# Filepath for TA, uncomment when submitting
filepath = ''

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [ ]:
import numpy as np
import pandas as pd
import time
import sklearn
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"sklearn version: {sklearn.__version__}")

Python version: 3.7.15
NumPy version: 1.21.6
Pandas version: 1.3.5
sklearn version: 1.0.2


## Reused functions from Assignment 1

In [ ]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

# FILTERING
def create_column_filter(df):

    # copy the input dataframe into df_copy
    df_copy = df.copy()

    for colname, values in df.iteritems():
        # only do something if it's not labeled 'CLASS' or 'ID'
        if colname == "CLASS" or colname == "ID":
          continue
        else:
            # Drop a column if it contains only missing values
            df_copy = df_copy.dropna(how = 'all', axis=1)

            # Drop a column if there's only one unique value...
            arr = df[colname].unique()
            # ... apart from the missing values
            arr = arr[~ pd.isnull(arr)] # '~' = np not operator (!), isnull = True where nan
            if arr.size == 1:
                #if colname in df_copy.columns: # this line replaced with errors='ignore' below
                df_copy = df_copy.drop([colname], axis=1, errors='ignore')
            #alternative apart from the missing values
            # elif arr.size == 2:
            #   for i in range(arr.size):
            #       if np.isnan(arr[i]):
            #           df_copy = df_copy.drop([colname],axis=1, errors='ignore')


    column_filter = df_copy.columns.values.tolist()

    return df_copy, column_filter

def apply_column_filter(df, column_filter):
    return df.loc[:,column_filter] # : for all rows , column_filter cols


#IMPUTATION
def create_imputation(df):
    dfcpy = df.copy()
    imputation = {}

    for col in df.columns:
        if col == "ID" or col == "CLASS":
            continue

        coltype = dfcpy[col].dtypes # get type of column

        if coltype == "int64" or coltype == "float64":
            # Replace nan values with mean of column for numeric cols
            mean = np.mean(dfcpy[col])

            # Rare case - all values nan
            if np.isnan(mean):
                mean = 0

            dfcpy[col] = dfcpy[col].fillna(mean)
            imputation[col] = mean

        if coltype == "object" or coltype == "category":
            # Replace nan values with mode (most common value) for object/catg columns
            mode = dfcpy[col].mode()

            # Rare case - all values nan (the category one in Hint 4 doesnt make sense
            # since category[0] = nan ...)
            if mode.empty:
                mode = ""
            else:
                mode = mode[0]

            dfcpy[col] = dfcpy[col].fillna(mode)
            imputation[col] = mode

    return dfcpy, imputation

def apply_imputation(df, imputation):
    dfcpy = df.copy()
    for col in df.columns:
        if col == "CLASS" or col == "ID":
            continue

        # Check if this col was imputed
        if col in imputation:
            value = imputation[col] # get imputation (mean or mode)
            dfcpy[col] = dfcpy[col].fillna(value)

    return dfcpy



#NORMALIZATION
def create_normalization(df, normalizationtype='minmax'):
    # copy the input dataframe into df_copy
    df_copy = df.copy()
    mapping = {}
    for colname, values in df_copy.iteritems():

        # only do something if it's not labeled 'CLASS' or 'ID'
        if colname == "CLASS" or colname == "ID":
          continue
        else :
            # Consider column of type "float" or "int"
            # Normalization of the value
            if df_copy[colname].dtype == 'float64' or 'int32' or 'int64' or 'float32':
                # dictionary from each column name to a triple ("minmax",min_value,max_value) or ("zscore",mean,std)

                if normalizationtype == "zscore":
                    # z normalization
                    mean = df_copy[colname].mean()
                    std = df_copy[colname].std()
                    #apply z normalization
                    df_copy[colname] = df_copy[colname].apply(lambda x: (x-mean)/std)
                    #filling the dictionary
                    triple = ["zscore", mean, std]
                    mapping[colname] = triple


                else :
                    #min-max normalization
                    min = df_copy[colname].min()
                    max = df_copy[colname].max()
                    # apply min-max normalization
                    df_copy[colname] = [(x-min)/(max-min) for x in df_copy[colname]]
                    # store max,mean and mean for each column
                    # for each column dictionary with three values (nested dictionary)
                    triple = ["minmax", min, max]
                    mapping[colname] = triple

    #print(df_copy)

    return df_copy, mapping

def apply_normalization(df, mapping):
    # copy the input dataframe into df_copy
    df_copy = df.copy()
    for colname, values in df_copy.iteritems():

        # only do something if it's not labeled 'CLASS' or 'ID'
        if colname == "CLASS" or colname == "ID":
          continue;
        else :
            # Consider column of type "float" or "int"
            # Normalization of the value
            if df_copy[colname].dtype == 'float64' or 'int32' or 'int64' or 'float32':

                if mapping.get(colname)[0] == 'zscore':
                    # z normalization
                    mean = mapping[colname][1]
                    std = mapping[colname][2]
                    #apply z normalization
                    df_copy[colname] = df_copy[colname].apply(lambda x: (x-mean)/std)

                else :
                    #min-max normalization
                    min = mapping[colname][1]
                    max = mapping[colname][2]


                    # apply min-max normalization
                    # limit the output range to [0,1]

                    df_copy[colname] = [1 if (x-min)/(max-min) > 1 else 0 if (x-min)/(max-min) < 0 else (x-min)/(max-min) for x in df_copy[colname]]


    return df_copy


#ONE-HOT
def create_one_hot(df):
    # copy the input dataframe into df_copy and create an empy dictionary
    df_copy = df.copy()
    one_hot = {}
    # only do something if the column is not labeled 'CLASS' or 'ID'
    for colname, values in df_copy.iteritems():
        if colname == "CLASS" or colname == "ID":
            continue
        else :
            # consider column of type "object" or "category"
            if df_copy[colname].dtype == 'object' or 'category':
                # get the values and put them in the dictionary
                one_hot[colname] = df_copy[colname].unique()
                # create new columns with the one_hot encoding
                for i, val in enumerate(values):
                    # ADDED str() below due to new error (val == float64)
                    df_copy[colname+"_"+str(val)] = df_copy[colname].apply(lambda x: 1 if x==val else 0)
                # drop the old column
                df_copy.drop(colname, axis=1, inplace=True)
    return df_copy, one_hot

def apply_one_hot(df, one_hot):
    # copy the input dataframe into df_copy
    df_copy = df.copy()
    for colname, values in df_copy.iteritems():
        for key, val in list(one_hot.items()):
            # only do something if the column is already in the dictionary
            if colname == key:
                #create new columns with the one_hot encoding
                for i, v in enumerate(val):
                    # ADDED str() below due to new error (val == float64)
                    df_copy[key+"_"+str(v)] = df_copy[key].apply(lambda x: 1 if x==v else 0)
                #drop the old column
                df_copy.drop(key, axis=1, inplace=True)
    return df_copy



#ACCURACY
def accuracy(df,correctlabels):

    preds = df.idxmax(axis=1) # get index(col) of max value on axis=1(row-wise)
    # print(correctlabels)
    # print(type(correctlabels))

    # print()
    # print(preds)
    # print(type(preds))

    correct = np.sum(preds == correctlabels) # get nr of equal values in two arrays

    acc = correct/len(correctlabels)

    return acc



#BRIER SCORE
def brier_score(df, correctlabels):
    #create matrix of zeros for true classes
    t_class = np.zeros((len(df), len(np.unique(correctlabels))))
    for colname, values in df.iteritems():
        #get the index of the true class
        ind = np.where(df.columns==colname)[0]
        for i, v in enumerate(correctlabels):
            # find the true class and change the values from 0 to 1
            if v == colname:
                t_class[i][ind] = 1
    # brier_score
    brier = np.mean(np.sum((df - t_class)**2, axis=1))
    return brier

# BINNING
def create_bins(df, nobins = 10, bintype="equal-width"):
    # copy the input dataframe into df_copy and create an empty dictionary
    df_copy = df.copy()
    binning = {}
    # only do something if the column is not labeled 'CLASS' or 'ID'
    for colname, values in df_copy.iteritems():
        if colname == "CLASS" or colname == "ID":
            continue;
        else :
            # only consider column of type "float" or "int"
            if df_copy[colname].dtype == 'float64' or 'int32' or 'int64' or 'float32':
                # create the categories for each value and the tresholds for the bins
                if bintype == "equal-width":
                    cat = pd.cut(df_copy[colname], bins=nobins, labels=False, retbins = True, duplicates = "drop")
                elif bintype == "equal-size":
                    cat = pd.qcut(df_copy[colname], q=nobins, labels=False, retbins = True, duplicates = "drop")
                # replace with the categories and change column type to "category"
                df_copy[colname] = cat[0].astype('category')
                # change the first and the last element of each binning to -np.inf and np.inf
                cat[1][0] = -np.inf
                cat[1][-1] = np.inf
                # add to dictionary
                binning[colname] = cat[1]
    return df_copy, binning



def apply_bins(df, binning):
    # copy the input dataframe into df_copy
    df_copy = df.copy()
    for colname, values in df_copy.iteritems():
       # only do something if the column is already in the dictionary
       for key, value in list(binning.items()):
            if colname == key:
                # create the category based on the bins provided in binning
                cat = pd.cut(df_copy[colname], bins=binning[colname], labels=False, retbins = False, duplicates = "drop")
                # replace with the categories and change type to "category"
                df_copy[colname] = cat.astype('category')
    return df_copy


#AUC
def auc(df,correctlabels):
    all_dicts = []

    # Create triple dictionaries for each class
    for col in df.columns:

        # Initialize dictionary with s_i --> [0,0]
        triples = {}
        for prob in df[col]:
            triples[prob] = [0,0]

        # Generate dictionary
        i = 0
        for prob in df[col]:
            # number of istances in that class
            if correctlabels[i] == col:
                triples[prob][0] += 1
            # number of istances not in that class
            else:
                triples[prob][1] += 1
            i+=1

        # print(f"Triples for {col}:")
        # print(triples)
        # print()
        all_dicts.append(triples)

    AUCs = [] # list for all AUCs

    # Calculate AUC for each triple dictionary
    for dicts in all_dicts:

        # Calculate total tps and fps
        tot_tp = 0
        tot_fp = 0
        # print("Probability ordering:")
        for prob in sorted(dicts,reverse=True):
            # print(prob)
            pair = dicts[prob]
            tot_tp += pair[0]
            tot_fp += pair[1]
        # print()

        # Begin AUC algorithm
        single_AUC = 0
        cov_tp = 0

        # Loop through the sorted pairs
        for prob in sorted(dicts,reverse=True):
            pair = dicts[prob]
            tp = pair[0]
            fp = pair[1]

            if fp == 0:
                cov_tp += tp
            elif tp == 0:
                single_AUC += (cov_tp/tot_tp) * (fp/tot_fp)
            else:
                single_AUC += (cov_tp/tot_tp) * (fp/tot_fp) + ((tp/tot_tp) * (fp/tot_fp))/2
                cov_tp += tp

        AUCs.append(single_AUC)

    # print("All AUCs:")
    # print(AUCs)
    # print()

    # Count how many times each col appears in correct labels
    col_correct = []
    for col in df.columns:
        correct_count = 0
        for label in correctlabels:
            if col == label:
                correct_count += 1
        # print(f"{col} is correct {correct_count} times")
        col_correct.append(correct_count)

    # Compute final weighted AUC
    AUC = 0
    for i in range(len(AUCs)):
        weight = col_correct[i]/len(correctlabels)
        AUC += weight * AUCs[i]

    # print()
    return AUC


#EUCLIDEAN DISTANCE
def eucledian(p1,p2):
    dist = np.sqrt(np.sum((p1-p2)**2))
    return dist

## 1. Define the class RandomForest

In [ ]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
class RandomForest:
# Input to __init__:
# self - the object itself
#
# Output from __init__:
# <nothing>
#
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None

# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#                      (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement)
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
    def generate_forest(self, no_trees, df):
        forest = []
        for i in range(no_trees):

            # Get bootstrap sample inidices
            rows = [i for i in range(len(df))]
            no_instances = len(rows)
            bootstrap_rows = np.random.choice(rows,size=no_instances,replace=True)

            # Separate Training instances, ID and CLASSES
            # Construct bootstrap samples
            bootstrap_classes = df['CLASS'].values
            bootstrap_classes = bootstrap_classes[bootstrap_rows]
            bootstrap_instances = df.drop('CLASS',axis=1).values
            bootstrap_instances = bootstrap_instances[bootstrap_rows,:]

            # Create a tree
            # Number of features
            no_features = int(np.log2(bootstrap_instances.shape[1]))
            tree = DecisionTreeClassifier(max_features=no_features)
            tree.fit(bootstrap_instances,bootstrap_classes)

            # Print tree
            #tree_desc = tree.export_text(tree, feature_names=list(df.columns))
            forest.append(tree)

        return forest

    def fit(self, df, no_trees=100):
        dfcpy = df.copy()

        # df_flitered : new df where columns containing only missing values have been dropped
        # col_filter : list of the names of the remaining columns
        df_filtered, col_filter = create_column_filter(dfcpy)
        self.column_filter = col_filter

        # df_imputed : new df missing numeric value in a column has been replaced by the mean of that column
        # missing categoric value in a column has been replaced by the mode of that column imputation
        # impt: mapping (dictionary) from column name to value that has replaced missing values
        df_imputed, impt = create_imputation(df_filtered)
        self.imputation = impt

        #df_onehot: new df where each categoric feature has been replaced by a set of binary features
        # one_hot: dictionary from column name to a set of categories (possible values for the feature)
        # is the hot encoding right because also numerical values are here encoded?
        df_onehot, one_h = create_one_hot(df_imputed)
        self.one_hot = one_h

        # list of class labels
        self.labels = sorted(dfcpy["CLASS"].unique())

        random_forest = self.generate_forest(no_trees,df_onehot)
        self.model = random_forest

# Input to predict:
# self - the object itself
# df   - a dataframe
#
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label,
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed
#         in the next part of the assignment.
    def predict(self,df):
        dfcpy = df.copy()

        df_filtered = apply_column_filter(dfcpy,self.column_filter)
        df_imputed = apply_imputation(df_filtered,self.imputation)
        df_onehot = apply_one_hot(df_imputed,self.one_hot)
        df_onehot = df_onehot.drop(["CLASS"],axis=1)

        data = df_onehot.values

        # Prediction matrix with test instance no of rows and class labels no of cols
        prediction_matrix = np.zeros((data.shape[0],len(self.labels)),dtype='float64')

        no_trees = 0
        for tree in self.model:
            # Prediction of tree has class labels x instances no of predictions
            pred = tree.predict_proba(data)

            prediction_matrix += pred
            no_trees += 1

        # Divide prediction matrix by number of trees
        prediction_matrix /= no_trees
        prediction_matrix = pd.DataFrame(prediction_matrix,columns=self.labels)

        return prediction_matrix

In [ ]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv(filepath+"tic-tac-toe_train.csv")

test_df = pd.read_csv(filepath+"tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 2.60 s.
Testing time: 0.06 s.
Accuracy: 0.9061
AUC: 0.9894
Brier score: 0.1780


In [ ]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

Accuracy on training set: 1.0000
AUC on training set: 1.0000
Brier score on training set: 0.0209


## 2a. Handling trees with non-aligned predictions

In [ ]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample.
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2);
#         this index points to which column in the prediction matrix the output column should be added to

class RandomForest:
# Input to __init__:
# self - the object itself
#
# Output from __init__:
# <nothing>
#
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None

    def generate_forest(self, no_trees, df):
        forest = []
        for i in range(no_trees):

            # Get bootstrap sample inidices
            rows = [i for i in range(len(df))]
            no_instances = len(rows)
            bootstrap_rows = np.random.choice(rows,size=no_instances,replace=True)

            # Separate Training instances, ID and CLASSES
            # Construct bootstrap samples
            bootstrap_classes = df['CLASS'].values
            bootstrap_classes = bootstrap_classes[bootstrap_rows]
            bootstrap_instances = df.drop('CLASS',axis=1).values
            bootstrap_instances = bootstrap_instances[bootstrap_rows,:]

            # Create a tree
            # Number of features
            no_features = int(np.log2(bootstrap_instances.shape[1]))
            tree = DecisionTreeClassifier(max_features=no_features)
            tree.fit(bootstrap_instances,bootstrap_classes)
            # print(tree.classes_)

            # Print tree
            #tree_desc = tree.export_text(tree, feature_names=list(df.columns))
            forest.append(tree)

        return forest

    def fit(self, df, no_trees=100):
        dfcpy = df.copy()

        # df_flitered : new df where columns containing only missing values have been dropped
        # col_filter : list of the names of the remaining columns
        df_filtered, col_filter = create_column_filter(dfcpy)
        self.column_filter = col_filter

        # df_imputed : new df missing numeric value in a column has been replaced by the mean of that column
        # missing categoric value in a column has been replaced by the mode of that column imputation
        # impt: mapping (dictionary) from column name to value that has replaced missing values
        df_imputed, impt = create_imputation(df_filtered)
        self.imputation = impt

        #df_onehot: new df where each categoric feature has been replaced by a set of binary features
        # one_hot: dictionary from column name to a set of categories (possible values for the feature)
        # is the hot encoding right because also numerical values are here encoded?
        df_onehot, one_h = create_one_hot(df_imputed)
        self.one_hot = one_h

       # list of class labels
        self.labels = sorted(dfcpy["CLASS"].unique())

        random_forest = self.generate_forest(no_trees,df_onehot)
        self.model = random_forest

    def predict(self,df):
        dfcpy = df.copy()

        df_filtered = apply_column_filter(dfcpy,self.column_filter)
        df_imputed = apply_imputation(df_filtered,self.imputation)
        df_onehot = apply_one_hot(df_imputed,self.one_hot)
        df_onehot = df_onehot.drop(["CLASS"],axis=1)

        data = df_onehot.values

        # create a mapping between n sorted class labels and their indices from 0 to n-1
        mapping = {}
        for index, label in enumerate(self.labels):
            mapping[label] = index

        # zero matrix test instance nr of rows and as column the class labels
        prediction_matrix = np.zeros((data.shape[0],len(self.labels)), dtype='float64')

        # go through all trees in the random forest
        no_trees = 0
        for t in self.model:
            # array with the probailities of each class appearing in the tree t
            proba = t.predict_proba(data)

            i = 0
            for index in range(len(self.labels)):
                #if the values in the labels and labels tree are equal
                # add to to the column with corresponding indice (from the mapping for this given class) the probability in this tree of this given glass
                # t.classes_ array of the classes present in t, sorted in the same order as t.predict_proba_
                if self.labels[index]==t.classes_[i]:
                    prediction_matrix[:,mapping[self.labels[index]]] += proba[:,i]
                    # go to the next value in the proba array and class tree array
                    i = i+1
                else:
                    prediction_matrix[:,mapping[self.labels[index]]] += 0

            # go to the next elements in class
            no_trees += 1

        # divide summed probabilities with no of trees
        prediction_matrix /= no_trees

        prediction_matrix = pd.DataFrame(prediction_matrix,columns=self.labels)

        return prediction_matrix

In [ ]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv(filepath+"anneal_train.csv")

test_df = pd.read_csv(filepath+"anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)


print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]
t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 5.08 s.
Testing time: 0.24 s.
Accuracy: 0.9354
AUC: 0.9746
Brier score: 0.1203


## 2b. Estimate predictive performance using out-of-bag predictions

In [ ]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively. By "aggregated out-of-bag predictions" is here meant the sum of all
#         predicted probabilities (one sum per class and instance). These sums should be divided by the number of predictions
#         (stored in the vector) in order to obtain a single class probability distribution per training instance.
#         This distribution is considered to be the out-of-bag prediction for each instance, and e.g., the class that
#         receives the highest probability for each instance can be compared to the correct label of the instance,
#         when calculating the accuracy using the out-of-bag predictions.
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector
#
#         For example, assuming that we have two class labels, then we may end up with the following matrix:
#
#         2 4
#         4 4
#         5 0
#         ...
#
#         and the vector (no. of predictions) (6, 8, 5, ...)
#
#         The resulting class probability distributions are:
#
#         0.333... 0.666...
#         0.5 0.5
#         1.0 0


class RandomForest:
# Input to __init__:
# self - the object itself
#
# Output from __init__:
# <nothing>
#
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
        self.obb_acc = None

    def generate_forest(self, no_trees, df):

        #create all lists and matrices needed
        forest = []
        oob_matrix = np.zeros((self.training_data.shape[0], len(self.labels)), dtype='float64')
        val_oob = np.zeros(self.training_data.shape[0])

        for t in range(no_trees):
            #get bootstrap sample indices
            rows = [t for t in range(len(df))]
            no_instances = len(rows)
            bootstrap_rows = np.random.choice(rows,size=no_instances,replace=True)

            #get out of bag sample indices
            oob_rows = []
            for i in range(len(df)):
                if i not in bootstrap_rows:
                    oob_rows.append(i)

            # Separate Training instances and CLASSES
            # Construct bootstrap samples
            bootstrap_classes = df['CLASS'].values
            bootstrap_classes = bootstrap_classes[bootstrap_rows]
            bootstrap_instances = df.drop('CLASS',axis=1).values
            bootstrap_instances = bootstrap_instances[bootstrap_rows,:]

            # Separate Training instances and CLASSES
            # Construct out of bag samples
            oob_instances = df.drop('CLASS',axis=1).values
            oob_instances = oob_instances[oob_rows,:]

            # Create a tree
            # Number of features
            no_features = int(np.log2(bootstrap_instances.shape[1]))
            tree = DecisionTreeClassifier(max_features=no_features)
            tree.fit(bootstrap_instances, bootstrap_classes)

            #for each oob instance predict the class
            for row in oob_rows:
                i = 0
                oob_pred = tree.predict_proba(oob_instances[i][None,:])
                #add a oob sample to the count
                val_oob[i] +=1

                for index in range(len(self.labels)):

                    #if the values in the labels and labels tree are the same, assign probability showed in oob_pred
                    if self.labels[index]==tree.classes_[i]:
                        oob_matrix[row, self.mapping[self.labels[index]]] += oob_pred[0, i]
                    #if the values in the labels and labels tree are different assign probability 0 to the labels
                    else:
                        oob_matrix[:, self.mapping[self.labels[index]]] += 0
                i = i+1

            forest.append(tree)

        #divide each element by the number of trees that it was built on
        for i in range(oob_matrix.shape[0]):
            for j in range(oob_matrix.shape[1]):
                if oob_matrix[i,j]!= 0 :
                    oob_matrix[i,j] /= val_oob[i]

        #create a dataframe and select the correctlabels
        oob_matrix = pd.DataFrame(oob_matrix, columns=self.class_val.unique())
        correctlabels = self.class_val

        #calculate accuracy
        self.oob_acc = accuracy(oob_matrix, correctlabels)

        return forest, self.oob_acc

    def fit(self, df, no_trees=100):
        dfcpy = df.copy()

        # df_flitered : new df where columns containing only missing values have been dropped
        # col_filter : list of the names of the remaining columns
        df_filtered, col_filter = create_column_filter(dfcpy)
        self.column_filter = col_filter

        # df_imputed : new df missing numeric value in a column has been replaced by the mean of that column
        # missing categoric value in a column has been replaced by the mode of that column imputation
        # impt: mapping (dictionary) from column name to value that has replaced missing values
        df_imputed, impt = create_imputation(df_filtered)
        self.imputation = impt

        #df_onehot: new df where each categoric feature has been replaced by a set of binary features
        # one_hot: dictionary from column name to a set of categories (possible values for the feature)
        # is the hot encoding right because also numerical values are here encoded?
        df_onehot, one_h = create_one_hot(df_imputed)
        self.one_hot = one_h

        # list of class labels
        self.labels = sorted(dfcpy["CLASS"].unique())

        #create y dataframe with classes
        self.class_val = df_onehot['CLASS']
        self.training_data = df_onehot

        #create mapping of
        self.mapping = {}
        for index, label in enumerate(self.labels):
            self.mapping[label] = index

        #create tghe forest with the function "generate_forest"
        random_forest, oob = self.generate_forest(no_trees,df_onehot)
        self.model = random_forest
        self.oob = oob

    def predict(self,df):
        dfcpy = df.copy()

        #pre-proccessing of the dataframe
        df_filtered = apply_column_filter(dfcpy,self.column_filter)
        df_imputed = apply_imputation(df_filtered,self.imputation)
        df_onehot = apply_one_hot(df_imputed,self.one_hot)
        df_onehot = df_onehot.drop(["CLASS"],axis=1)

        data = df_onehot.values

        # zero matrix as rows number of test instances and as column the class labels
        prediction_matrix = np.zeros((data.shape[0],len(self.labels)), dtype='float64')

        no_trees = 0
        for t in self.model:
            #predict on the test data
            proba = t.predict_proba(data)
            i = 0
            for index in range(len(self.labels)):
                #if the values in the labels and labels tree are the same, assign probability showed in proba
                if self.labels[index]==t.classes_[i]:
                    prediction_matrix[:,self.mapping[self.labels[index]]] += proba[:,i]
                    i = i+1
                #if the values in the labels and labels tree are different assign probability 0 to the labels
                else:
                    prediction_matrix[:,self.mapping[self.labels[index]]] += 0
            # go to the next elements in class
            no_trees += 1

        prediction_matrix /= no_trees
        prediction_matrix = pd.DataFrame(prediction_matrix,columns=self.labels)

        return prediction_matrix

In [ ]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv(filepath+"anneal_train.csv")

test_df = pd.read_csv(filepath+"anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))
test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:117: RuntimeWarning: divide by zero encountered in double_scalars


Training time: 6.81 s.
OOB accuracy: 0.7550
Testing time: 0.24 s.
Accuracy: 0.9354
AUC: 0.9757
Brier score: 0.1192


In [ ]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:117: RuntimeWarning: divide by zero encountered in double_scalars


Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.02


### 2b for tic-tac-toe

If implemented on a dataset based on exercise 1, the algorithm provides a better OOB accuracy.

In [ ]:
class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
        self.obb_acc = None


    def generate_forest(self, no_trees, df):
        forest = []
        oob_matrix = np.zeros((self.training_data.shape[0], len(self.labels)), dtype='float64')
        val_oob = np.zeros(self.training_data.shape[0])

        for t in range(no_trees):

            # Get bootstrap sample inidices
            rows = [t for t in range(len(df))]
            no_instances = len(rows)
            bootstrap_rows = np.random.choice(rows,size=no_instances,replace=True)

            oob_rows = []
            for i in range(len(df)):
                if i not in bootstrap_rows:
                    oob_rows.append(i)

            # Separate Training instances and CLASSES
            # Construct bootstrap samples
            bootstrap_classes = df['CLASS'].values
            bootstrap_classes = bootstrap_classes[bootstrap_rows]
            bootstrap_instances = df.drop('CLASS',axis=1).values
            bootstrap_instances = bootstrap_instances[bootstrap_rows,:]

            oob_instances = df.drop('CLASS',axis=1).values
            oob_instances = oob_instances[oob_rows,:]

            # Create a tree
            # Number of features
            no_features = int(np.log2(bootstrap_instances.shape[1]))
            tree = DecisionTreeClassifier(max_features=no_features)
            tree.fit(bootstrap_instances, bootstrap_classes)

            #the predictions can be added directly to the rows of the matrix since
            #all the classes are in every tree
            oob_pred = tree.predict_proba(oob_instances)
            ind = 0
            for i in oob_rows:
                oob_matrix[i,:] += oob_pred[ind]
                val_oob[i] +=1
                ind +=1

            forest.append(tree)

        for row in range(oob_matrix.shape[0]):
            oob_matrix[row,:] /=  val_oob[row]


        oob_matrix = pd.DataFrame(oob_matrix, columns=self.class_val.unique())
        correctlabels = self.class_val

        self.oob_acc = accuracy(oob_matrix, correctlabels)

        return forest, self.oob_acc

    def fit(self, df, no_trees=100):
        dfcpy = df.copy()

        # df_flitered : new df where columns containing only missing values have been dropped
        # col_filter : list of the names of the remaining columns
        df_filtered, col_filter = create_column_filter(dfcpy)
        self.column_filter = col_filter

        # df_imputed : new df missing numeric value in a column has been replaced by the mean of that column
        # missing categoric value in a column has been replaced by the mode of that column imputation
        # impt: mapping (dictionary) from column name to value that has replaced missing values
        df_imputed, impt = create_imputation(df_filtered)
        self.imputation = impt

        #df_onehot: new df where each categoric feature has been replaced by a set of binary features
        # one_hot: dictionary from column name to a set of categories (possible values for the feature)
        # is the hot encoding right because also numerical values are here encoded?
        df_onehot, one_h = create_one_hot(df_imputed)
        self.one_hot = one_h

       # list of class labels
        self.labels = sorted(dfcpy["CLASS"].unique())

        onehot_labels = []
        for label in dfcpy["CLASS"]:
            if label == 'positive':
                onehot_labels.append(1)
            else:
                onehot_labels.append(0)
        df_onehot['CLASS'] = onehot_labels

        self.class_val = df_onehot['CLASS']
        self.training_data = df_onehot
        #print(self.training_data)

        random_forest, oob = self.generate_forest(no_trees,df_onehot)
        self.model = random_forest
        self.oob = oob


    def predict(self,df):
        dfcpy = df.copy()


        df_filtered = apply_column_filter(dfcpy,self.column_filter)
        df_imputed = apply_imputation(df_filtered,self.imputation)
        df_onehot = apply_one_hot(df_imputed,self.one_hot)
        df_onehot = df_onehot.drop(["CLASS"],axis=1)

        self.test_data = df_onehot.values

        mapping = {}
        for index, label in enumerate(self.labels):
            mapping[label] = index

        prediction_matrix = np.zeros((self.test_data.shape[0],len(self.labels)),dtype='float64')

        no_trees = 0
        for tree in self.model:
            # Prediction of tree has class labels no of predictions
            pred = tree.predict_proba(self.test_data)
            #print(np.shape(pred))
            prediction_matrix += pred
            no_trees += 1

        prediction_matrix /= no_trees
        prediction_matrix = pd.DataFrame(prediction_matrix,columns=self.labels)

        return prediction_matrix

In [ ]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv(filepath+"tic-tac-toe_train.csv")

test_df = pd.read_csv(filepath+"tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))
test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 2.87 s.
OOB accuracy: 0.9207
Testing time: 0.07 s.
Accuracy: 0.9102
AUC: 0.9905
Brier score: 0.1787


In [ ]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.02
